In [ ]:
import pandas as pd
from datetime import datetime
import numpy as np

FILE_CATEGORY_TREE = '../data/category_tree.csv'
FILE_EVENTS = '../data/events.csv'
FILE_ITEM_PROPERTIES_1 = '../data/item_properties_part1.csv'
FILE_ITEM_PROPERTIES_2 = '../data/item_properties_part2.csv'
FILE_ITEM_PROPERTIES_ALL = '../data/item_properties_all.csv'

pd.set_option('mode.chained_assignment',None)

from sklearn import model_selection
from sklearn.linear_model import LogisticRegression

### Original EDA analysis

``` python
category_tree = pd.read_csv(FILE_CATEGORY_TREE)

category_tree.head()

category_tree.shape

category_tree[category_tree.parentid.isnull()].sort_values('categoryid')

def find_root(x, df):
    while True: 
        if np.isnan(df.loc[df.categoryid == x, 'parentid'].unique()[0]):
            return x
        else:
            x = df.loc[df.categoryid == x, 'parentid'].unique()[0]

l = []
for c in category_tree.categoryid:
    l.append(find_root(c,category_tree))

category_tree = pd.concat([category_tree,pd.Series(l,name='top_parent')],axis=1)

top_parent_summary = category_tree.groupby('top_parent')['categoryid'].count().reset_index()
top_parent_summary.rename(columns = {'categoryid':'num_categories'}, inplace=True)
top_parent_summary.sort_values('num_categories',ascending=False)

top_parent_summary.shape

25 top-level categories 

events = pd.read_csv(FILE_EVENTS)

events.head()

events.describe(include='all')

events.info()

events.event.value_counts()

events.itemid.unique().shape

235,061 products view/addtocart/transcation

def convert_to_local(x):
    return datetime.fromtimestamp(x/1000)

events['local_date_time'] = events.timestamp.apply(convert_to_local)

events[events.visitorid == 1150086].sort_values('local_date_time').head(10)

events.sort_values(['visitorid','local_date_time'], inplace=True)
events['time_diff'] = events.groupby('visitorid')['timestamp'].diff(periods=-1) *-1

events.time_diff = events.time_diff / 1000 # convert from milliseconds to seconds

events.groupby('visitorid')['time_diff'].agg(['mean','count']).reset_index().sort_values('count', ascending=False).head()

events[events.visitorid == 280150].sort_values('local_date_time').head(10)

events.agg({'local_date_time':['min','max']})

item_properties = pd.read_csv(FILE_ITEM_PROPERTIES_1)

item_properties['local_date_time'] = item_properties.timestamp.apply(convert_to_local)

item_properties[item_properties.itemid.isin([133542])].sort_values('timestamp')

item_properties_2 = pd.read_csv(FILE_ITEM_PROPERTIES_2)

item_properties_2['local_date_time'] = item_properties_2.timestamp.apply(convert_to_local)

item_properties_2[item_properties_2.itemid.isin([133542])].sort_values('timestamp')

item_properties_master = item_properties.append(item_properties_2)

item_properties_master[item_properties_master.itemid.isin([133542])].sort_values(['property','local_date_time'])

item_properties_master[item_properties_master.itemid.isin([167873])].sort_values(['property','local_date_time'])

item_property_unique = item_properties_master.loc[:,['itemid','property']].drop_duplicates()

property_count = item_property_unique.groupby('property')['itemid'].count().sort_values(ascending=False).reset_index()

property_count[property_count.itemid == 417053]

Above properties are represented of all items. Let's see what the unique values are for these properties.

item_properties_master.loc[item_properties_master.property == '364','value'].value_counts().sort_values(ascending=False).reset_index().head(10)

item_properties_master[item_properties_master.value.str.contains(r'[^\s]')]
```

### New EDA Analysis

In [ ]:
events = pd.read_csv('../data/events.csv')
events.local_date_time = pd.to_datetime(events.local_date_time)

# set a row number primary key
# events['pk'] = events.index

In [ ]:
category_tree = pd.read_csv('../data/category_tree_parent.csv')

In [ ]:
item_properties_master = pd.read_csv('../data/item_properties_master.csv')
item_properties_master.local_date_time = pd.to_datetime(item_properties_master.local_date_time)

### Reduce data set size and begin to build MVP feature set

|visitorid|counts|
---|---|
|152963 |    2054
|994820  |   1661
|1150086  |  1524
|247235    | 1425
|645525     |1411
|79627      |1180
|530559     |1091
|737053     |1055

## Feature building

1. Calcualte session id for the events
2. Calcualte number of views in each session
3. Calcualte total session length
4. Build design matrix

In [ ]:
## Calculate Session ID

def calc_session_id(df, mask):
    df['session_id'] = np.nan
    ind = df.groupby('visitorid').head(1).index
    df.loc[ind, 'session_id'] = 1

    count_session = df[mask].shape[0] + 1
    df.loc[mask, 'session_id'] = np.arange(2,count_session+1)

    # fill in all of the gaps
    df.session_id.fillna(method = 'ffill', inplace=True)

    # make the session id unique
    df.session_id = df.visitorid.astype(str) + '_' + df.session_id.astype(int).astype(str)
    
    return df

def convert_to_local(x):
    return datetime.fromtimestamp(x/1000)

# get data
events = pd.read_csv('../data/events.csv')
events['local_date_time'] = events.timestamp.apply(convert_to_local)

# reduce data set size for MVP
events_trimmed = events[events.local_date_time >= datetime(2015, 8, 15)]

print(f'Total number of events: {events_trimmed.shape[0]:,}')
print()

# first calcualte sessions for each buy transaction
events_trimmed.sort_values(['visitorid','local_date_time'], inplace=True)
events_trimmed['prev_event'] = events_trimmed.groupby('visitorid').event.shift(1)
sub = (events_trimmed.event == 'view') & (events_trimmed.prev_event == 'transaction')
events_trimmed = calc_session_id(events_trimmed, sub)

print(f'Total number of sessions dividing on transaction: {len(events_trimmed.session_id.unique()):,}')
print()

# calcualte the time diff within each session
events_trimmed.sort_values(['session_id', 'local_date_time'], inplace = True)

events_trimmed['time_diff'] = (events_trimmed
                               .groupby('session_id')['timestamp']
                               .diff(1)
                               .fillna(0) 
                               / 1000)

# events_trimmed.['time_diff'] = events_trimmed.time_diff / 1000
events_trimmed['page_length'] = events_trimmed.groupby('visitorid').time_diff.shift(-1)

# re-calaculate sessions with a new session starting whenever a buy occurs or if a view lasts longer than 3.5 minutes
session_time_limit = 600 ## Should consider building a time limit per user, instead of global

sub = (((events_trimmed.event == 'view') 
       & (events_trimmed.prev_event == 'transaction'))
      | ((events_trimmed.event == 'view')
        & (events_trimmed.time_diff > session_time_limit)))

events_trimmed = calc_session_id(events_trimmed, sub)

print(f'Total number of sessions adding a time limit: {len(events_trimmed.session_id.unique()):,}')
print()

## Calcualte total views per session

view_counts_df = (events_trimmed[events_trimmed.event == 'view']
                     .groupby('session_id')['event']
                     .count())
view_counts_df.name = 'view_count'

## Calcualte total session length per session

session_length_df = (events_trimmed
                        .groupby('session_id')['page_length']
                        .agg(['sum','mean']))
session_length_df.rename(columns = {'sum':'session_length','mean':'avg_len_per_pg'}, inplace=True)

## Build Design Matrix

events_trimmed['buy_event'] = 0
events_trimmed.loc[events_trimmed.event == 'transaction', 'buy_event'] = 1

buy_event = events_trimmed.groupby('session_id')['buy_event'].max()

print(f'Buy event observations: {len(buy_event):,}')
print()

design_df = pd.concat([buy_event, view_counts_df, session_length_df],axis=1, sort=True)
design_df = design_df.fillna(0)

print(f'Design observations: {design_df.shape[0]:,}')
print()

print(f'Precentage of buys: {len(design_df[design_df.buy_event >= 1].buy_event) / len(design_df.buy_event):.2%}')

In [ ]:
events_trimmed[events_trimmed.visitorid == 152963]
# .session_id.fillna(method = 'ffill')

## 5 mins per page - _Using this setting_
Total number of events: 599,871

Total number of sessions dividing on transaction: 337,455

Total number of sessions adding a time limit: 415,948

Buy event observations: 415,948

Design observations: 415,948

Precentage of buys: 0.93%

## 3.5 mins per page
Total number of events: 599,871

Total number of sessions dividing on transaction: 337,455

Total number of sessions adding a time limit: 444,864

Buy event observations: 444,864

Design observations: 444,864

Precentage of buys: 0.87%


## 10 mins per page
Total number of events: 599,871

Total number of sessions dividing on transaction: 337,455

Total number of sessions adding a time limit: 404,052

Buy event observations: 404,052

Design observations: 404,052

Precentage of buys: 0.96%

In [ ]:
print(design_df.shape)
print(len(buy_event))

In [ ]:
events_trimmed[events_trimmed.visitorid == 1000675]

In [ ]:
y = design_df.buy_event
X = design_df.drop(columns='buy_event')
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size = 0.2, random_state = 1234)

In [ ]:
log_model = LogisticRegression()
log_model.fit(X_train, y_train)

In [ ]:
log_model.score(X_train, y_train)

In [ ]:
y_train_pred = log_model.predict(X_train)

In [ ]:
from sklearn.metrics import f1_score

In [ ]:
f1_score(y_train, y_train_pred)

In [ ]:
property_df = item_properties_master[item_properties_master.property == 'categoryid']
property_df.head()

In [ ]:
events_trimmed.sort_values('local_date_time', inplace=True)
property_df.sort_values('local_date_time', inplace=True)

print(f'events_trimmed rows: {events_trimmed.shape[0]:,}')
events_trimmed_property = pd.merge_asof(events_trimmed, property_df, on="local_date_time", by="itemid") 
print(f'merged rows: {events_trimmed_property.shape[0]:,}')

In [ ]:
events_trimmed_property.dropna(subset=['value']).shape


In [ ]:
## Make this into a function so I can get any property
def set_item_property(property_master_df, feature_df, prop, property_name ):
    # filter down to the property
    property_df = property_master_df[property_master_df.property == prop]

    # get all property rows for each item in the feature data set
    merge_event_property_df = feature_df.merge(property_df, how='left',on='itemid')
    print(f'Rows when left join {merge_event_property_df.shape[0]:,}')

    merge_event_property_df = feature_df.merge(property_df, how='inner',on='itemid')
    print(f'Rows when inner join {merge_event_property_df.shape[0]:,}')

    print('Using inner join for now, and will come back later.')

    # remove all rows where the property was updated after the event timestamp
    print(f'Nuber of items: {len(merge_event_property_df.itemid.)}')
    merge_event_property_df = (merge_event_property_df[merge_event_property_df.local_date_time_x 
                                                       > merge_event_property_df.local_date_time_y])

    property_max_date_time = (merge_event_property_df
                              .groupby(['local_date_time_x','itemid','session_id'])['local_date_time_y']
                              .max()
                              .reset_index())
    
    merge_event_property_df = (merge_event_property_df
                               .merge(property_max_date_time
                                      , how='inner'
                                      , on=['local_date_time_x','itemid','session_id','local_date_time_y']))
    print(f'Rows when inner join {merge_event_property_df.shape[0]:,}')

    # clean up the df
    drop_c = ['timestamp_y', 'property', 'local_date_time_y']
    merge_event_property_df.drop(columns=drop_c, inplace=True)

    rename_c = {'timestamp_x':'timestamp', 'local_date_time_x':'local_date_time','value':property_name}
    merge_event_property_df.rename(columns=rename_c, inplace=True)

    return merge_event_property_df, property_max_date_time

In [ ]:

events_trimmed[events_trimmed.visitorid == 152963]
# print(events_trimmed[(events_trimmed.visitorid == 152963) & (events_trimmed.time_diff > 210)].shape)

In [ ]:
210 / 60

In [ ]:
events_trimmed.time_diff.describe()

In [ ]:
events_trimmed = events[events.local_date_time >= datetime(2015, 8, 15)]

print(f'Trimmed events {events_trimmed.shape[0]:,}')
print()

print(f'Count of events: \n{events_trimmed.event.value_counts()}')
print()
# unique visitors
print(f'Visitors that bought something {events_trimmed[events_trimmed.event == "transaction"].visitorid.unique().shape[0]:,}')
print()

###### Probably remove this as we want to look at buy vs not buy

# # all visitors where at least one session ended in a transaction
# visitors = events_trimmed[events_trimmed.event == 'transaction'].visitorid.unique()
# print(f'Events for visitors who bought something {events_trimmed[events_trimmed.visitorid.isin(visitors)].shape[0]:,}')
# buy_visitors = events_trimmed[events_trimmed.visitorid.isin(visitors)]

######

# calculate the session_id
# session_id identifes each pattern of view...transaction for each visitor as unique
l = []
for v in events_trimmed.visitorid.unique():
    v_df = events_trimmed[events_trimmed.visitorid == v].sort_values('local_date_time')
    prev_event = 'view'
    session_id = 1
    for i in v_df.index:
        if prev_event == 'transaction' and v_df.loc[i,'event'] != 'transaction':
            session_id += 1

        prev_event = v_df.loc[i,'event']
        l.append(session_id)

# assign each session_id and make it unique
events_trimmed['session_id'] = l
events_trimmed.session_id = events_trimmed.visitorid.astype('str') + '_' + events_trimmed.session_id.astype('str')

###### Probably remove this as we want to look at buy vs not buy

# # group by session_id and remove those sessions without a transaction
# grouped_events = buy_visitors.groupby(['session_id','event'])['visitorid'].count().reset_index()
# valid_sessions = grouped_events[grouped_events.event == 'transaction'].session_id
# buy_visitors = buy_visitors[buy_visitors.session_id.isin(valid_sessions)]

# print()
# print(f'Sessions that have at least one transaction {buy_visitors.shape[0]:,}')

######

# calcaulte session length feature
events_trimmed.time_diff = events_trimmed.time_diff.shift(1)
events_trimmed.rename(columns={'time_diff':'session_length'}, inplace=True)

# calculate hour of day and day of week
events_trimmed['session_hour'] = events_trimmed.local_date_time.dt.hour
events_trimmed['session_dow'] = events_trimmed.local_date_time.dt.dayofweek

# set the category id property
# print()
# print('Adding category')
# buy_visitors, _ = set_item_property(item_properties_master, buy_visitors, 'categoryid', 'category_id' )
# print(f'Updated shape of the feature DF {buy_visitors.shape[0]:,}')

# set the available property
# print()
# print('Adding available')
# buy_visitors, property_group_df = set_item_property(item_properties_master, buy_visitors, 'available', 'available' )
# print(f'Updated shape of the feature DF {buy_visitors.shape[0]:,}')

In [ ]:
buy_visitors_category.head()

In [ ]:
# buy_visitors.timestamp.value_counts().head()
buy_visitors[buy_visitors.timestamp == '143991494102']

In [ ]:
from itertools import chain, combinations

def key_options(items):
    return chain.from_iterable(combinations(items, r) for r in range(1, len(items)+1) )

df = buy_visitors

# iterate over all combos of headings, excluding ID for brevity
for candidate in key_options(list(df)):
    deduped = df.drop_duplicates(candidate)

    if len(deduped.index) == len(df.index): #and len(deduped.index) <= 4:
        print(','.join(candidate))

In [ ]:
buy_visitors_category.merge(buy_visitors_category_avail, how='left', on=['timestamp', 'visitorid','itemid','event']).shape

In [ ]:
property_group_df.head()

In [ ]:
(buy_visitors
    .merge(property_group_df
              , how='inner'
              , on=['local_date_time_x','itemid','session_id','local_date_time_y']))

In [ ]:
item_properties_master.property.value_counts()
# TODO Available and then figure out the t-1 thing.
# Thinking of adding a feature to start with transaction and count backwards within each session ordered by time